In [ ]:
%pip install git+https://github.com/ultralytics/ultralytics.git

In [ ]:
!cd ultralytics
%pip install -e .

# 1. Load a model, train it, evaluate its performance on a validation set, and export it to ONNX format

In [ ]:
from ultralytics import YOLO

# Create a new YOLO model from scratch
model = YOLO("yolov8n.yaml")

# Load a pretrained YOLO model (recommended for training)
model = YOLO("yolov8n.pt")

# Train the model using the 'coco8.yaml' dataset for 3 epochs
results = model.train(data="coco8.yaml", epochs=3)
# Resume training the model from the last checkpoint
#model = YOLO("last.pt")
#results = model.train(resume=True)

# Evaluate the model's performance on the validation set
# Validate on training data
results = model.val()
# Validate on separate data
# model.val(data="path/to/separate/data.yaml")

# Perform object detection on an image using the model
results = model("https://ultralytics.com/images/bus.jpg")

# Export the model to ONNX format
success = model.export(format="onnx")

# 2. Explorer datasets with advanced semantic, vector-similarity and SQL search

In [ ]:
# Use Images
from ultralytics import Explorer

# create an Explorer object
exp = Explorer(data="coco8.yaml", model="yolov8n.pt")
exp.create_embeddings_table()

similar = exp.get_similar(img="https://ultralytics.com/images/bus.jpg", limit=10)
print(similar.head())

# Search using multiple indices
similar = exp.get_similar(
    img=["https://ultralytics.com/images/bus.jpg", "https://ultralytics.com/images/bus.jpg"], limit=10
)
print(similar.head())

In [ ]:
# Using Dataset Indices
from ultralytics import Explorer

# create an Explorer object
exp = Explorer(data="coco8.yaml", model="yolov8n.pt")
exp.create_embeddings_table()

similar = exp.get_similar(idx=1, limit=10)
print(similar.head())

# Search using multiple indices
similar = exp.get_similar(idx=[1, 10], limit=10)
print(similar.head())

# 3. Customize Trainers to support custom tasks

In [ ]:
# Detection Trainer Example
from ultralytics.models.yolo import DetectionPredictor, DetectionTrainer, DetectionValidator

# trainer
trainer = DetectionTrainer(overrides={})
trainer.train()
trained_model = trainer.best

# Validator
val = DetectionValidator(args=...)
val(model=trained_model)

# predictor
pred = DetectionPredictor(overrides={})
pred(source=SOURCE, model=trained_model)

# resume from last weight
overrides["resume"] = trainer.last
trainer = detect.DetectionTrainer(overrides=overrides)

# 4. Callbacks 允许用户在流程的特定点执行自定义功能，从而增强和修改工作流程

1. 训练器回调函数: on_pretrain_routine_start, on_pretrain_routine_end, on_train_start, on_train_epoch_start, on_train_batch_start, optimizer_step, on_before_zero_grad, on_train_batch_end, on_train_epoch_end, on_fit_epoch_end, on_model_save, on_train_end, on_params_update, teardown

2. 验证器回调函数: on_val_start, on_val_batch_start, on_val_batch_end, on_val_end

3. 预测器回调函数: on_predict_start, on_predict_batch_start, on_predict_postprocess_end, on_predict_batch_end, on_predict_end

4. 导出器回调函数: on_export_start, on_export_end

In [2]:
# 记录自定义指标: 在训练或验证的每个时期结束时记录额外的指标.
# 数据增强: 在预测或训练批次期间实现自定义数据转换或增强.
# 保存中间结果

from ultralytics import YOLO

def on_predict_batch_end(predictor):

  """将预测结果与对应帧合并; 修改预测器结果."""

  _, image, _, _ = predictor.batch

  image = image if isinstance(image, list) else [image]

  predictor.results = zip(predictor.results, image)


model = YOLO("yolov8n.pt")
# 注册回调, 事件名称为"on_predict_batch_end", 回调函数为"on_predict_batch_end"  类似hook机制, iptables 4表5链, git hook
model.add_callback("on_predict_batch_end", on_predict_batch_end)


for result, frame in model.predict():

  # 或 model.track()

  pass

100%|██████████| 6.25M/6.25M [03:09<00:00, 34.5kB/s]

WARNING ⚠️ 'source' is missing. Using 'source=/Users/es/miniconda3/envs/ultralytics/lib/python3.10/site-packages/ultralytics/assets'.



image 1/2 /Users/es/miniconda3/envs/ultralytics/lib/python3.10/site-packages/ultralytics/assets/bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 122.4ms


ValueError: not enough values to unpack (expected 4, got 3)